# Cleaning notebook

* [Imports](#imports)
* [Dataset loading](#dataset-loading)
* [Features filtering](#feature-filtering)
    * [FR and threshold filtering](#fr-and-threshold-filtering)
    * [Amount of missing values](#amount-of-missing-values)
    * [Correlations between missing values](#correlations-between-missing-values)
    * [Correlations between features](#correlations-between-features)
* [Row filtering](#row-filtering)
    * [Empty rows](#empty-rows)
    * [Wrong values](#wrong-values)
    * [Duplicated rows](#duplicated-rows)
* [Missing values](#missing-values)
    * [Zero Imputation](#zero-imputation)
    * [Label Fusion](#label-fusion)
* [ML imputation](#ml-imputation)
    * [KKN imputation](#knn-imputation)
    * [RandomForest Regressor](#randomforest-regressor)
    * [Classifier imputation](#classifier-imputation)
    * [Actual imputation](#actual-imputation)
* [Outliers](#outliers)
* [Saving](#saving)

<a name="imports"></a>
## Imports

In [ ]:
import os
import re
from typing import List

import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

<a name="dataset-loading"></a>
## Dataset loading

In [ ]:
if not os.path.exists('fr.openfoodfacts.org.products.csv.zip'):
    !wget https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip

In [ ]:
if not os.path.exists('dataset.csv'):
    !unzip fr.openfoodfacts.org.products.csv.zip
    !mv fr.openfoodfacts.org.products.csv dataset.csv
!head -n 3 dataset.csv

In [ ]:
if 'df' not in locals():
    df = pd.read_csv('./dataset.csv', sep='\t')
print(df.info(verbose=True))
df.head()

<a name="features-filtering"></a>
## Features filtering

<a name="fr-and-threshold-filtering"></a>
### FR and threshold filtering

* Keep only the products sold in France
* Keep certain columns (the interesting and not empty ones)

In [ ]:
def fr_filter(s: str) -> bool:
    try:
        s_part = re.split('; |, | :', s.lower())
        return 'fr' in s_part or 'france' in s_part
    except:
        return False
        
def column_filter(columns: List[str], paterns_to_keep: List[str]=None):
    filtered_columns = []
    if paterns_to_keep is None:
        paterns_to_keep = ['image', 'name', 'category', '_100g', 'grade', 'score', 'group']
    for column in columns:
        for patern_to_keep in paterns_to_keep:
            if patern_to_keep in column:
                filtered_columns.append(column)
                break
    return filtered_columns

In [ ]:
df_fr = df[df["countries"].apply(fr_filter)]
del df

df_fr = msno.nullity_filter(df_fr, 'top', 0.01)
df_fr_filtered = df_fr[['code'] + column_filter(df_fr.columns.values)]
del df_fr

df_fr_filtered.head()

<a name="amount-of-missing-values"></a>
### Amount of missing values

In [ ]:
msno.matrix(df_fr_filtered)

In [ ]:
msno.bar(df_fr_filtered)

<a name="correlations-between-missing-values"></a>
### Correlations between missing values

In [ ]:
msno.heatmap(df_fr_filtered)

In [ ]:
msno.dendrogram(df_fr_filtered)

In [ ]:
df_fr_filtered.isna().sum()

<a name="correlations-between-features"></a>
### Correlations between features

In [ ]:
print((df_fr_filtered['nutrition-score-fr_100g'] != df_fr_filtered['nutrition-score-uk_100g']).sum())
print((df_fr_filtered['nutrition-score-fr_100g'].isna() != df_fr_filtered['nutrition-score-uk_100g'].isna()).sum())

In [ ]:
sns.relplot(data=df_fr_filtered, x="nutrition-score-fr_100g", y="nutrition-score-uk_100g")

In [ ]:
print((df_fr_filtered['nutrition-score-fr_100g'].isna() != df_fr_filtered['nutrition_grade_fr'].isna()).sum())
print((df_fr_filtered['nutrition-score-fr_100g'].isna() != df_fr_filtered['energy_100g'].isna()).sum())

In [ ]:
sns.catplot(data=df_fr_filtered, x="nutrition-score-fr_100g", y="nutrition_grade_fr", kind="box")

<a name="row-filtering"></a>
## Row filtering

<a name="empty-rows"></a>
### Empty rows

* Based on numerical features

In [ ]:
numerical_columns = column_filter(df_fr_filtered.columns.values, paterns_to_keep=['_100g'])
print(df_fr_filtered[numerical_columns].isna().all(axis='columns').sum())
print(df_fr_filtered[numerical_columns[:-2]].isna().all(axis='columns').sum())
print(df_fr_filtered[numerical_columns[1:-2]].isna().all(axis='columns').sum())
mask = df_fr_filtered[numerical_columns[1:-2]].isna().all(axis='columns')

In [ ]:
df_fr_filtered = df_fr_filtered[~mask]

<a name="wrong-values"></a>
### Wrong values

* Based on numerical features

In [ ]:
print((df_fr_filtered[numerical_columns[:-2]] < 0).any(axis='columns').sum())
mask = (df_fr_filtered[numerical_columns[:-2]] < 0).any(axis='columns')

In [ ]:
df_fr_filtered = df_fr_filtered[~mask]

In [ ]:
print((df_fr_filtered[numerical_columns[1:-2]] > 100).any(axis='columns').sum())
mask = (df_fr_filtered[numerical_columns[1:-2]] > 100).any(axis='columns')
df_fr_filtered.loc[mask, ["product_name"]+numerical_columns]

In [ ]:
df_fr_filtered = df_fr_filtered[~mask]

In [ ]:
print((df_fr_filtered['energy_100g'] > 3765.6).sum())#900 Kcal/100g = 3765.6kj/100g
mask = df_fr_filtered['energy_100g'] > 3765.6

In [ ]:
df_fr_filtered = df_fr_filtered[~mask]

* Included columns

In [ ]:
print((df_fr_filtered['carbohydrates_100g']<df_fr_filtered['sugars_100g']).sum())
sub_fat = df_fr_filtered[column_filter(df_fr_filtered.columns.values, paterns_to_keep=['ed_fat_100g'])].sum(axis='columns')
print((df_fr_filtered['fat_100g']<sub_fat).sum())
mask = (df_fr_filtered['carbohydrates_100g']<df_fr_filtered['sugars_100g'])

In [ ]:
df_fr_filtered = df_fr_filtered[~mask]

<a name="duplicated-rows"></a>
### Duplicated rows

In [ ]:
print(df_fr_filtered["code"].duplicated().sum())
print(df_fr_filtered[df_fr_filtered.columns.values[1:]].duplicated().sum())
mask = df_fr_filtered[df_fr_filtered.columns.values[1:]].duplicated()

In [ ]:
df_fr_filtered = df_fr_filtered[~mask]

<a name="missing-values"></a>
## Missing values

<a name="zero-imputation"></a>
### Zero Imputation

* Composition missing elements are replaced by 0

In [ ]:
imputed_columns = [c for c in numerical_columns if 'nutrition-score' not in c and 'energy' not in c]
df_fr_filtered.loc[:, imputed_columns] = df_fr_filtered[imputed_columns].fillna(0)
df_fr_filtered.isna().sum() 

In [ ]:
df_desc = df_fr_filtered.describe()
df_desc

<a name="feature-fusion"></a>
### Features Fusion

* Fusion of the similar features
* Name imputation (by the group)

In [ ]:
print((df_fr_filtered['product_name'].isna() & ~df_fr_filtered['generic_name'].isna()).sum())
mask = df_fr_filtered['product_name'].isna() & ~df_fr_filtered['generic_name'].isna()

In [ ]:
df_fr_filtered.loc[mask, 'product_name'] = df_fr_filtered.loc[mask, 'generic_name']
df_fr_filtered.drop(columns=['generic_name'], inplace=True)

In [ ]:
(df_fr_filtered['main_category'].isna() & ~df_fr_filtered['main_category_fr'].isna()).sum()

In [ ]:
df_fr_filtered.drop(columns=['main_category_fr'], inplace=True)

In [ ]:
(df_fr_filtered['image_url'].isna() & ~df_fr_filtered['image_small_url'].isna()).sum()

In [ ]:
df_fr_filtered.drop(columns=['image_small_url'], inplace=True)

In [ ]:
print(df_fr_filtered[['pnns_groups_1','pnns_groups_2','main_category']].isna().sum())
print(df_fr_filtered[['pnns_groups_1','pnns_groups_2','main_category']].nunique())
print((df_fr_filtered['pnns_groups_2']=="unknown").sum())

In [ ]:
print((df_fr_filtered['pnns_groups_1']=="unknown").sum())
print((df_fr_filtered['pnns_groups_2']=="unknown").sum())
print(((df_fr_filtered['pnns_groups_2']=="unknown") & (df_fr_filtered['pnns_groups_1']!="unknown")).sum())

In [ ]:
df_fr_filtered['pnns_groups_2'] = df_fr_filtered['pnns_groups_2'].fillna(value="unknown")

In [ ]:
print((df_fr_filtered['pnns_groups_2'].isna() & ~df_fr_filtered['pnns_groups_1'].isna()).sum())
print((df_fr_filtered['pnns_groups_2'].isna() & ~df_fr_filtered['main_category'].isna()).sum())

In [ ]:
df_fr_filtered.drop(columns=['pnns_groups_1','main_category'], inplace=True)

In [ ]:
print((df_fr_filtered['product_name'].isna() & ~df_fr_filtered['pnns_groups_2'].isna() & (df_fr_filtered['pnns_groups_2']!="unknown")).sum())
mask = df_fr_filtered['product_name'].isna() & ~df_fr_filtered['pnns_groups_2'].isna() & (df_fr_filtered['pnns_groups_2']!="unknown")

In [ ]:
df_fr_filtered.loc[mask, 'product_name'] = df_fr_filtered.loc[mask, 'pnns_groups_2']

<a name="ml-imputation"></a>
## ML imputation

* Recover the nutrition scores (numerical) and the nutrition grade (categorical)

The two features are linked but not directly. The numerical feature needs a regression and the categorical needs a classification

In [ ]:
bool_index = (df_fr_filtered['nutrition-score-fr_100g'].notna() & df_fr_filtered['energy_100g'].notna())

X_num = np.array(df_fr_filtered.loc[bool_index, numerical_columns])
y_clf = np.array(df_fr_filtered.loc[bool_index, 'nutrition_grade_fr'])

X_num_train, X_num_test, y_clf_train, y_clf_test = train_test_split(X_num, y_clf, test_size=0.2, random_state=42)

In [ ]:
def reg_metrics(y_true, y_pred):
    metric_list = ['mean_squared_error', 'mean_absolute_error', 'r2_score']
    for metric_name in metric_list:
        score = eval(f"{metric_name}(y_true, y_pred)")
        print(f"{metric_name}: {score:.3f}")

In [ ]:
def clf_metrics(y_true, y_pred):
    metric_list = ['accuracy', 'balanced_accuracy', 'f1', 'precision', 'recall']
    kwargs = {'accuracy':{}, 'balanced_accuracy':{}, 'f1':{'average':'macro'}, 'precision':{'average':'macro'}, 'recall':{'average':'macro'}}
    for metric_name in metric_list:
        score = eval(f"{metric_name}_score(y_true, y_pred, **kwargs[metric_name])")
        print(f"{metric_name}: {score:.3f}")

<a name="knn-imputation"></a>
### KKN imputation

Imputer based on a KNN imputer.

In [ ]:
%%time
imputer = KNNImputer(n_neighbors=3, weights='distance')
imputer.fit(X_num_train)

In [ ]:
%%time
X_num_test_knn = np.copy(X_num_test)
X_num_test_knn[:,(0,-2,-1)] = np.nan
y_pred_knn = imputer.transform(X_num_test_knn)[:,(0,-2,-1)]

In [ ]:
reg_metrics(X_num_test[:,(0,-2,-1)], y_pred_knn)

<a name="randomforest-regressor"></a>
### RandomForest Regressor

In [ ]:
%%time
regr = RandomForestRegressor(n_estimators=20, max_depth=20, random_state=0)
regr.fit(X_num_train[:,1:-2], X_num_train[:,(0,-2,-1)])

In [ ]:
reg_metrics(X_num_train[:,(0,-2,-1)], regr.predict(X_num_train[:,1:-2]))

In [ ]:
importances = regr.feature_importances_
std = np.std([tree.feature_importances_ for tree in regr.estimators_], axis=0)
forest_importances = pd.Series(importances, index=numerical_columns[1:-2])
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
%%time
y_pred_rfg = regr.predict(X_num_test[:,1:-2])

In [ ]:
reg_metrics(X_num_test[:,(0,-2,-1)], y_pred_rfg)

<a name="classifier-imputationn"></a>
### Classifier imputation

In [ ]:
%%time
clf = RandomForestClassifier(n_estimators=20, max_depth=20, random_state=0, class_weight='balanced')
clf.fit(X_num_train[:,1:-2], y_clf_train)

In [ ]:
clf_metrics(y_clf_train, clf.predict(X_num_train[:,1:-2]))

In [ ]:
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
forest_importances = pd.Series(importances, index=numerical_columns[1:-2])
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
%%time
y_pred_rfc = clf.predict(X_num_test[:,1:-2])

In [ ]:
clf_metrics(y_clf_test, y_pred_rfc)

In [ ]:
%%time
clf_full = RandomForestClassifier(n_estimators=20, max_depth=20, random_state=0, class_weight='balanced')
clf_full.fit(X_num_train, y_clf_train)

In [ ]:
clf_metrics(y_clf_train, clf_full.predict(X_num_train))

In [ ]:
importances = clf_full.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf_full.estimators_], axis=0)
forest_importances = pd.Series(importances, index=numerical_columns)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
%%time
y_pred_rfc_full = clf_full.predict(X_num_test)

In [ ]:
clf_metrics(y_clf_test, y_pred_rfc_full)

In [ ]:
df_fr_filtered['nutrition_grade_fr'].isna().sum()

<a name="actual-imputation"></a>
### Actual imputation

In [ ]:
bool_index = (df_fr_filtered['nutrition-score-fr_100g'].isna() | df_fr_filtered['energy_100g'].isna())
df_fr_filtered.loc[bool_index, ['energy_100g','nutrition-score-fr_100g', 'nutrition-score-uk_100g']] = regr.predict(np.array(df_fr_filtered.loc[bool_index, numerical_columns])[:,1:-2])

In [ ]:
bool_index = df_fr_filtered['nutrition_grade_fr'].isna()
df_fr_filtered.loc[bool_index, 'nutrition_grade_fr'] = clf_full.predict(np.array(df_fr_filtered.loc[bool_index, numerical_columns]))

In [ ]:
df_imputed_desc = df_fr_filtered.describe()
df_imputed_desc

In [ ]:
diff_desc = df_imputed_desc - df_desc
diff_desc[['energy_100g','nutrition-score-fr_100g', 'nutrition-score-uk_100g']]/df_desc.loc['std',['energy_100g','nutrition-score-fr_100g', 'nutrition-score-uk_100g']]

<a name="outliers"></a>
## Outliers

In [ ]:
tau = 5
upper_bound = df_imputed_desc.loc['mean',:] + tau * df_imputed_desc.loc['std',:]
lower_bound = df_imputed_desc.loc['mean',:] - tau * df_imputed_desc.loc['std',:]

In [ ]:
print((df_fr_filtered < lower_bound).any(axis='columns').sum())
print((df_fr_filtered > upper_bound).any(axis='columns').sum())

In [ ]:
test_feature = 'fiber_100g'

upper_bound_en = df_imputed_desc.loc['mean', test_feature] + tau * df_imputed_desc.loc['std', test_feature]
lower_bound_en = df_imputed_desc.loc['mean', test_feature] - tau * df_imputed_desc.loc['std', test_feature]

In [ ]:
print((df_fr_filtered[test_feature] < lower_bound_en).sum())
print((df_fr_filtered[test_feature] > upper_bound_en).sum())

<a name="saving"></a>
## Saving

In [ ]:
print(df_fr_filtered['product_name'].isna().sum())
df_fr_filtered = df_fr_filtered[df_fr_filtered['product_name'].notna()]

In [ ]:
df_fr_filtered.to_csv('cleaned_dataset.csv', index=False)